In [1]:
import os

def delete_videos(dir_path):
    for root, _, files in os.walk(dir_path):
        for file in files:
            if file.endswith(".mp4"):
                os.remove(os.path.join(root, file))

dir_path = "/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/experiments/result-hidden/num_frames_ablations/nba_3k_train_bal_test_imb"
delete_videos(dir_path)

In [1]:
from format_data_kinetics_tf import clean_directory, format_dataset_kinetics
from split_dataset import copy_and_split_dataset

dst_dir = "/mnt/opr/levlevi/contextualized-shot-quality-analysis/data/experiments/result-hidden/balance_ablation/meg_result_hidden_nba_5k_train_balanced_test_imbalanced_4s"
# test_dir = "/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/experiments/test-sets/result-hidden/nba_.5k_imbalanced_4s"

# copy_and_split_dataset(
#     train_val_dir="/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/experiments/result-hidden/nba_results_hidden_26.7k_480",
#     test_dir=test_dir,
#     dst_dir=dst_dir,
#     num_files_to_copy=3000,
#     # class_balance=[0.4529, 0.5471],
# )

clean_directory(dst_dir)
format_dataset_kinetics(dst_dir)

100%|██████████| 5499/5499 [00:16<00:00, 338.32it/s]


In [4]:
import os

fp_made = "/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/experiments/result-hidden/num_frames_ablations/nba_3k_train_bal_test_imb/train/made"
fp_missed = "/playpen-storage/levlevi/contextualized-shot-quality-analysis/data/experiments/result-hidden/num_frames_ablations/nba_3k_train_bal_test_imb/train/missed"
num_made = os.listdir(fp_made).__len__()
num_missed = os.listdir(fp_missed).__len__()
num_made, num_missed

num_made, num_missed
num_made / (num_made + num_missed)

0.49831144465290805

In [3]:
import pandas as pd

ex_fp = '/mnt/opr/levlevi/contextualized-shot-quality-analysis/data/mixed-leagues/game-logs/Game_log_data/22-23/2_Euroleague/938086_Olimpia Milano - Real Madrid.csv'
df = pd.read_csv(ex_fp, delimiter=';')
df

,,,,,,,,,,,id,match_date,tournament_id,tournament_name,season_id,season_name,round_id,round_name,team1_id,team1_name,team2_id,team2_name,team1_score,team2_score,status_id,status_name,arena_id,arena_name,match_name,ts
247965418,100100,Ball in play,NaN,NaN,69.0,Real Madrid,NaN,NaN,NaN,NaN,NaN,NaN,1,79.101,14.01,7.51,1.0,Start,69.0,Real Madrid,1.0,0.0,10.96,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-03 21:59:44.174534
247972120,200900,Assisting,13692.0,Walter Tavares,69.0,Real Madrid,NaN,NaN,NaN,NaN,1366.0,Dzanan Musa,1,90.061,NaN,NaN,NaN,NaN,69.0,Real Madrid,1.0,0.0,10.96,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-03 23:26:01.909146
247965420,800201,2+,13692.0,Walter Tavares,69.0,Real Madrid,26461.0,Brandon Davies,2910.0,Olimpia Milano,1366.0,Dzanan Musa,1,90.061,26.34,6.97,100.0,End,69.0,Real Madrid,1.0,0.0,10.96,Pick-n-roll without a ball,Right,Alley oop,NaN,NaN,Contested,2022-11-03 21:59:44.309061
247972121,200300,Pick'n'Roll,13692.0,Walter Tavares,69.0,Real Madrid,1004.0,Devon Hall,2910.0,Olimpia Milano,1366.0,Dzanan Musa,1,90.061,NaN,NaN,2.0,Possession,69.0,Real Madrid,1.0,0.0,10.96,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-03 23:26:02.035089
247965419,400101,Accurate pass,1366.0,Dzanan Musa,69.0,Real Madrid,NaN,NaN,NaN,NaN,13692.0,Walter Tavares,1,89.961,NaN,NaN,2.0,Possession,69.0,Real Madrid,1.0,0.0,10.96,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-03 21:59:44.184544
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247971774,1200200,Second chance,NaN,NaN,2910.0,Olimpia Milano,NaN,NaN,NaN,NaN,NaN,NaN,4,511.239,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-03 23:23:11.138831
247971789,1800600,Timeout,NaN,NaN,2910.0,Olimpia Milano,NaN,NaN,NaN,NaN,NaN,NaN,4,552.695,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-03 23:23:11.936372
247972088,1600200,Power forward,9685.0,Billy Baron,2910.0,Olimpia Milano,13087.0,Nazareth Mitrou-Long,2910.0,Olimpia Milano,NaN,NaN,4,584.615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-03 23:25:19.75014
247972301,1600100,Center,6079.0,DeShaun Thomas,2910.0,Olimpia Milano,5468.0,Johannes Voigtmann,2910.0,Olimpia Milano,NaN,NaN,4,905.789,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2022-11-03 23:28:23.182458
